In [ ]:
#Reading the dataset(only for kernel!!)
library("readr")
train <- read.csv("../input/Train_UWu5bXk.csv")
test <- read.csv("../input/Test_u94Q5KV.csv")

In [ ]:
#Viewing first few rows of the training dataset
head(train)

In [ ]:
#Understanding the internal structure of the  training dataset.
str(train)
## Notice that the variables like Item_Type are already categorical(factor, not character), so 
## there is no need to change them
## We have 7 categorical variables, 4 numerical variables and 1 integer variable.

In [ ]:
#Understanding the internal structure of the  test dataset.
str(test)
## Notice that the variables like Item_Type are already categorical(factor, not character), so 
## there is no need to change them.
## We see that there is no Item_Outlet-Sales in the test data. That is the variable that we
## have to predict.

In [ ]:
#Summarizing the numeric variables
summary(train)
## Here, take note of the fact that the number of NA(missing) values is also shown

In [ ]:
#Summarizing variable-wise(here Outlet_Establishment)
summary(train$Outlet_Establishment)

In [ ]:
#Understanding the categorical variable distribution.(here Outlet_Type)
table(train$Outlet_Type)

In [ ]:
#Checking the number of NA(missing) values
table(train$Outlet_Type, useNA = "always")
## Here the number of missing values is 0

In [ ]:
#Checking the distribution of categorical variables percentage wise
prop.table(table(train$Outlet_Type, useNA = "always"))*100

In [ ]:
## Explore the data, check out every variable in the data to get more comfortable
## with it(fill '---" with the desired variable). You can also check out the test data

In [ ]:
#Numeric Variables
#summary(train$---)

In [ ]:
#Categorical Variables
#table(train$---, useNA = "always")
#prop.table(table(train$---, useNA = "always"))*100

In [ ]:
## We have to perform same operations on train and test data. Rather than doing the same
## operations twice, it is better to append(attach) the train and test data together.

In [ ]:
#Appending the train and test dataset

## We can't append the datasets unless they have the same number of columns. Therefore, we will
## add another column(Item_outlet_Sales)(the dependent variable), to the test dataset.
test$Item_Outlet_Sales <- NA

## Now, we will append the datasets, and create a new dataset named 'data'
data <- rbind(train, test)

In [ ]:
# Checking the dimensions of data
dim(data)
## Let's compare to the test and train datasets
dim(train)
dim(test)

In [ ]:
#Checking the missing(NA) values of the data
summary(is.na(data)) 
## is.na() returns a logical vector which indicates the number of datapoints which are missing
## TRUE indicates missing.

In [ ]:
## We can see that only Item_Weight has missing values.
## Item_Outlet_Sales has missing values because it is the dependent variable, and the data points 
## corresponding to the test data have missing values. So we don't have to worry about that.

In [ ]:
#Missing imputing values.
#Method-1(Mean method)
## We input the mean of rest of the data in place of the missing values.

#Finding the mean of the variable
 mean(data$Item_Weight) 
## will return NA. therefore, we have to specify to find the mean the 'non-NA' values
 mean((data$Item_Weight), na.rm = TRUE)
## na.rm = TRUE will remove the NA values to while calculating the mean. 
## OR
 mean(data$Item_Weight[!is.na(data$Item_Weight)])
## '!' returns the FALSE values

In [ ]:
## Now, we will imsert this value in place of the missing value
## We will make another dataset to observe.
data_mean_impu <- data
data_mean_impu$Item_Weight[is.na(data_mean_impu$Item_Weight)] <- mean(data$Item_Weight[!is.na(data_mean_impu$Item_Weight)])
summary(data_mean_impu$Item_Weight)

In [ ]:
#Method-2 (Median imputation)
## We input the median of rest of the data in place of the missing values.
 median((data$Item_Weight), na.rm = TRUE)
 median(data$Item_Weight[!is.na(data$Item_Weight)])

In [ ]:
## Now, we will imsert this value in place of the missing value
## We will make another dataset to observe.
data_median_impu <- data
data_median_impu$Item_Weight[is.na(data_median_impu$Item_Weight)] <- median(data$Item_Weight[!is.na(data_median_impu$Item_Weight)])
summary(data_median_impu$Item_Weight)

In [ ]:
##Since the only variable with missing values is numerical, we won't be using the k-NN method,
##which is for categorical variables. However, you can have a glance at k-NN implementation
##  https://www.youtube.com/watch?v=u8XvfhBdbMw 

In [ ]:
#Method-3(Advanced Packages)(Optional)
## There are many powerful packages in R which help us input missing values
##For e.g. Mice, missForest, mi, Amelia etc. All packages have their specific uses,
## and pitfalls. For, more information, you can refer:-
## https://www.analyticsvidhya.com/blog/2016/03/tutorial-powerful-packages-imputing-missing-values/


In [ ]:
## First, we need to understand the distriution of Item_Weight. We can understand it better,
## if we can visually see it. Here, we will plot the histogram.
hist(data$Item_Weight)
hist(data_mean_impu$Item_Weight)
hist(data_median_impu$Item_Weight)

In [ ]:
## As we can see, the methods of imputing mean and median for this case 
## would be a bad idea(the data biases too much). Instead,we will use the package Hmisc.
## But first, we will do some data cleaning.
## Notice that in Item_Fat_Content, we have LF, low fat and Low Fat, which are same. Also,
## reg and Regular are same. Therefore, add the observations of "LF" and "low fat" to "Low fat",
## and "reg" to "Regular"
data$Item_Fat_Content <- gsub("LF", "Low Fat", data$Item_Fat_Content)
data$Item_Fat_Content <- gsub("low fat", "Low Fat", data$Item_Fat_Content)
data$Item_Fat_Content <- gsub("reg", "Regular", data$Item_Fat_Content)
table(data$Item_Fat_Content) # Viewing the variable

In [ ]:
## Notice that Item_Type has factors which are not food items. So, Item_Fat_Content makes no
## sense. Hence we will add a new factor(level): "None", which will correspond to the 
## non-food items in Item_Type.

#Adding new level in Item_Fat_Content "None"
levels(data$Item_Fat_Content) <- c(levels(data$Item_Fat_Content), "None")

## Based on Item_Type, for "health and Hygiene", "Household" and "Others",
## we will change the Item_Fat_Content factor to "None".
data[which(data$Item_Type=="Health and Hygiene"), ]$Item_Fat_Content = "None"
data[which(data$Item_Type=="Household"), ]$Item_Fat_Content = "None"
data[which(data$Item_Type=="Others"), ]$Item_Fat_Content = "None"
data$Item_Fat_Content <- as.factor(data$Item_Fat_Content)
table(data$Item_Fat_Content) # Viewing the variable

In [ ]:
## Notice that the column Outlet_Size has blank values. We will now procede to replace them.
## We will compare them with other variables, to understand better about the missing variables.

In [ ]:
Outlet_Size_Length <- as.data.frame(setNames(aggregate(data$Outlet_Size, 
                              by=list(Category=data$Outlet_Identifier, 
                                      Category=data$Outlet_Type,
                                      Category=data$Outlet_Location_Type,
                                      Category=data$Outlet_Size), FUN= length), 
                                      c("Outlet_Identifier","Outlet_Type", "Outlet_Location_Type", "Outlet_Size", "number")))
Outlet_Size_Length # Viewing the data frame

In [ ]:
table(data[which(data$Outlet_Type=='Grocery Store'), ]$Outlet_Size, useNA='always')
table(data[which(data$Outlet_Type=='Supermarket Type1'), ]$Outlet_Size, useNA='always')
table(data[which(data$Outlet_Type=='Supermarket Type2'), ]$Outlet_Size, useNA='always')
table(data[which(data$Outlet_Location_Type=='Tier 1'), ]$Outlet_Size, useNA='always')
table(data[which(data$Outlet_Location_Type=='Tier 2'), ]$Outlet_Size, useNA='always')
table(data[which(data$Outlet_Location_Type=='Tier 3'), ]$Outlet_Size, useNA='always')

In [ ]:
## As per analysis, we would be better off by fixing category 'Small', on every 'Grocery Store',
## and for the remaining blank values, we will assign category 'Small' again.
data[which(data$Outlet_Identifier=="OUT010"), ]$Outlet_Size <- "Small"
data[which(data$Outlet_Identifier=="OUT017"), ]$Outlet_Size <- "Small"
data[which(data$Outlet_Identifier=="OUT045"), ]$Outlet_Size <- "Small"
data$Outlet_Size <- factor(data$Outlet_Size) # Running factor on an existing factor variable will reset the levels to only those that are present.
table(data$Outlet_Size)

In [ ]:
## Since we are only concerned with how old the outlet is, and not the establishment year,
## we will substitute Outlet_Establishment_Year with Outlet_Year
data$Outlet_Year <- as.factor(2013-data$Outlet_Establishment_Year) # Since this is 2013 data
table(data$Outlet_Year)

In [ ]:
## Visualizing Item_MRP with ggplot
library(ggplot2)
ggplot(data, aes(Item_MRP)) + geom_density(adjust = 1/5)
## It is obvious that we would be better off by converting Item_MRP to Categorical variable

In [ ]:
data$MRP_Factor <- as.factor(ifelse(data$Item_MRP < 70, "Low" ,
                             ifelse(data$Item_MRP <130, "Medium",
                             ifelse(data$Item_MRP < 201, "High", "Very High"))))
table(data$MRP_Factor) # Viewing the new variable

In [ ]:
## Notice that Item_Visibility has a minimum value of 0. It seems absurd that an item has 0 
## visibility. Therefore, we will modify that column.
data_Visibility <- data[which(data$Item_Visibility !=0), c(1,4)]
IV_m <- aggregate(Item_Visibility~Item_Identifier, data=data_Visibility, FUN=mean)

In [ ]:
## Now, we will substitute the values according to IV_m in Item_Visibility

In [ ]:
for(i in 1:1559){
    if(dim(data[which(data$Item_Identifier==IV_m[i, ]$Item_Identifier & data$Item_Visibility=='0'), ])[1]!='0'){
            data[which(data$Item_Identifier==IV_m[i, ]$Item_Identifier & data$Item_Visibility=='0'), ]$Item_Visibility <- IV_m[i, ]$Item_Visibility
    }
}

In [ ]:
summary(data$Item_Visibility) #Checking

In [ ]:
## If you look at Item_Identifier, i.e. the unique ID of each item, it starts with either 
## FD, DR or NC. If you see the categories, these look like being Food, Drinks
## and Non-Consumables. So, we will create a new broad variable that assigns these three factors
data$Item_Identifier <- as.character(data$Item_Identifier)
data$Item_Type_Broad <- as.factor(ifelse(grepl('FD',data$Item_Identifier)== 'TRUE',"Foods",
                            ifelse(grepl('DR',data$Item_Identifier)== 'TRUE',"Drinks",
                            "Non-Consumables" )))
data$Item_Identifier <- as.factor(data$Item_Identifier)

In [ ]:
table(data$Item_Type_Broad, useNA='always') #Checking

In [ ]:
## We will aso create a more brief Item_Type_Broad2 from Item_Type
data$Item_Type_Broad2 <- as.factor(ifelse(data$Item_Type == 'Baking Goods' | data$Item_Type == 'Breads', "Bakery", 
                                          ifelse(data$Item_Type == 'Canned' | data$Item_Type == 'Frozen Foods' | data$Item_Type == 'Dairy', "Refrigerated",
                                          ifelse(data$Item_Type == 'Meat' | data$Item_Type == 'Seafood', "Non-Veg", 
                                          ifelse(data$Item_Type == 'Household' | data$Item_Type == 'Others' | data$Item_Type == 'Health and Hygiene', "Other",
                                          ifelse(data$Item_Type == 'Hard Drinks', "Alcoholic",
                                          ifelse(data$Item_Type == 'Snack Foods' | data$Item_Type == 'Soft Drinks', 'Snacks',
                                          ifelse(data$Item_Type == 'Breakfast', 'Breakfast', "NOTA"))))))))
table(data$Item_Type_Broad2, useNA = 'always')

In [ ]:
## Now we will input missing values in Item_Weight
set.seed(825) #Setting seed
library(Hmisc)
imputed_arg <- aregImpute(~Item_Weight+Item_Fat_Content+Item_Type+Item_Type_Broad+ Item_Type_Broad2, 
                          data = data[ ,c(2,3,5,15,16)], boot.method = 'approximate bayesian', n.impute = 1)

In [ ]:
for(i in 1:2439){
    if(dim(data[which(is.na(data$Item_Weight) == TRUE), ])[1]!='0'){
            data[which(is.na(data$Item_Weight) == TRUE), ]$Item_Weight <- imputed_arg$imputed$Item_Weight
    }
}

In [ ]:
summary(data$Item_Weight) #Checking

In [ ]:
## Now we will attempt to remove the outliers in variables. There are many ways to do that.
## We will check the inter-quartile range.
boxplot(train$Item_Visibility)
IQR(data$Item_Visibility) # Inter-Quartile Range
UL <- (median(data$Item_Visibility) + (1.5*IQR(data$Item_Visibility)))
LL <- (median(data$Item_Visibility) - (1.5*IQR(data$Item_Visibility)))
length(data[which(data$Item_Visibility > UL | data$Item_Visibility < LL), ]$Item_Visibility)
ggplot(data, aes(Item_Visibility)) + geom_density(adjust = 1/5)
## We can see there are a lot of points, and the curve is severely skewed (right skew)(Actually,
## they are not outliers. Rather, we percieve them to be so, because we have assumed the data
## to be of gaussian distribution).When we replace outliers with median or mean,
## the skew(ness) changes. Since there are a lot of points, it would be unwise to do that.
## We can use several alorithms to correct that, but for the time being, we will chose to 
## ignore the outliers. Also, earlier, we have handled the 'intuitive' outliers, 
##the 0s in Item_Visibility. However, if you do want to input median you can use the code,
## data[which(data$Item_Visibility > UL | data$Item_Visibility < LL), ]$Item_Visibility <- median(data$Item_Visibility)

In [ ]:
## We are done with the data cleaning and feature engineering. If you can come up with more ideas,
## you are welcome to try.
## Now, we will divide the data into train and test.

In [ ]:
# Dividing data into train and test
train <- data[1:8523, ]
test <- data[8524:14204, ]


In [ ]:
## lets fit regression model 
lmfit <- lm(Item_Outlet_Sales~., data = train[-c(1,6,7,8)])
summary(lmfit)
par(mfrow=c(2,2))
plot(lmfit)

#Adjusted R-squared value is 0.5409
#Residual plot seems funnel shaped suggest our model is suffering from heteroskedasticity 
#Cook distance is used to observe the influencial outliers
##QQ plot is used to validate the assumption of normal distribution of dataset
#lets tackle this by taking log of response variable



In [ ]:
#lets tackle this by taking log of response variable
lmfit2<- lm(log(Item_Outlet_Sales) ~ ., data = train[-c(1,6,7,8)])
summary(lmfit2)
par(mfrow=c(2,2))
plot(lmfit2)

#new adjusted R^2 value increased significantly and model is no longer suffering from non constant variance


 


In [ ]:
#Lets check RMSE values
library(Metrics)
rmse(train$Item_Outlet_Sales, lmfit$fitted.values) 
#RMSE value is 1154.04 (model which is suffering from heteroskedasticity)
rmse(train$Item_Outlet_Sales, exp(lmfit2$fitted.values))
#New RMSE value is 1134.95 Thats Awesome! we improved a lot.

In [ ]:
#Prediction on test_dataset
options(warn=-1)#Just to remove unnecessary warnings
log_test_outlet_sale <- predict(lmfit2,test[-c(1,6,7,8,12)])
# The predict will return log value of outlet_sale for test dataset
#Hence the real value can be obtained by taking exponent of the log_test_outlet_sale
test_outlet_sale <-exp(log_test_outlet_sale)
test_outlet_sale[1:10]#(have a look on first 10 values of outlet sale on test dataset)
#You can submit the following output in the competition to get an idea of your standing



In [ ]:
#The code given below is only for reference *Only to demonstrate how cross validation os implementated in R

### Model Building with cross validation ###
library(caret)
#Setting all the parameters repeated_cv *five fold *repeated three times
rf.label<-train$Item_Outlet_Sales
cv.5.folds <- createMultiFolds(rf.label, k = 5, times = 3)
ctrl.1 <- trainControl(method = "repeatedcv", number = 5, repeats = 10, index = cv.5.folds)
options(warn=-1)
lm.5.cv.1 <- train(x = train[-c(1,6,7,8,12)], y = rf.label, method = "lm", trControl = ctrl.1,metric='RMSE')
lm.5.cv.1
test_outlet_sale_cv <- predict(lm.5.cv.1,test[-c(1,6,7,8,12)])



In [ ]:
## Now we will perform ridge and lasso regression
library(glmnet)
library(SparseM)
set.seed(999)
# Ridge Regression
demo_sample <- data.frame(model.matrix(~.-1,train[,-c(1,6,7,8,12)]))

x<-as.matrix(demo_sample)

y <- train$Item_Outlet_Sales
#head(x)
grid =10^ seq (10,-2, length =100)
ridge.mod =cv.glmnet(x,y,alpha =0, lambda =grid)
coef(ridge.mod) #Penalise the coefficient of less important variables
bestlam =ridge.mod$lambda.min#Selecting best lambda value
ridge.pred=predict (ridge.mod ,s=bestlam ,newx=x) #predicted value of ridge model corresponding to lambda returning lowest RMSE
sqrt(mean(( ridge.pred -train$Item_Outlet_Sales)^2) ) #Root mean square error value *can be again reduced by proper feature engineering
dim(coef(ridge.mod))
plot(ridge.mod)
#ridge.plot(y, x, lambda = seq(0, 2, by = 0.1) )

In [ ]:
# Lasso Regression  (Least Absolute Shrinkage Selector Operator)
lasso.mod =cv.glmnet(x,y,alpha =1, lambda =grid)
coef(lasso.mod)# *.* represent lasso reduces the weights to zero unlike ridge
bestlamda =lasso.mod$lambda.min#Selecting best lambda value
lasso.pred=predict (lasso.mod ,s=bestlamda ,newx=x) #predicted value of ridge model corresponding to lambda returning lowest RMSE
sqrt(mean(( lasso.pred -train$Item_Outlet_Sales)^2) ) #Root mean square error value *can be again reduced by proper feature engineering
dim(coef(lasso.mod))
plot(lasso.mod)